<a href="https://colab.research.google.com/github/Crawnicles/Algo-trading-project/blob/main/VolatilityBreakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project setup
Configure imports and setup the connection to the Alpaca API

In [ ]:
# Install packages not included with Colab
!pip install python_dotenv
!pip install alpaca_trade_api
!pip install backtrader

# Import Packages
from alpaca_trade_api.rest import TimeFrame
import alpaca_trade_api as tradeapi
from alpaca_trade_api.stream import Stream
import backtrader as bt
import matplotlib
import pandas as pd
from alpaca_trade_api import TimeFrameUnit
from pandas.core.frame import DataFrame
import numpy as np

# Alpaca API info

Required information needed to connect to the Alpaca API.

In [4]:
ALPACA_API_KEY = 'PKZ7UQDMAWO5UAULDXCO'
ALPACA_SECRET_KEY = 'ESBhzVmXVPtUjbzdmKx3airlPQ5eBHlbamnNy123'

alpaca_api = tradeapi.REST(ALPACA_API_KEY,
                ALPACA_SECRET_KEY, 
                'https://paper-api.alpaca.markets',
                'v2')

# Configure Base algorithim

# Algorithim
Creates a function that takes in a Dataframe, calculates the price range and marks if the close goes above or below that range to indicate a Buy or Sell. This returns a Data frame that can be used to plot data.

In [24]:
def VolBreakout(PD_Dataframe, percentage: int):
  # Convert int to Percentage
  multiplier = percentage / 100

  # Calculate the Top and Bottom of the range to determine where positions will be taken
  top_range_calc = PD_Dataframe['high'][-1] * multiplier
  bottom_range_calc = PD_Dataframe['low'][-1] * multiplier
  top_of_range = PD_Dataframe['open'] + top_range_calc
  bottom_of_range = PD_Dataframe['open'] - bottom_range_calc

  # Initiate a Column that counts the number of times a cross happens on above the range and below the range
  PD_Dataframe['Top of Range'] = round(top_of_range, 2)
  PD_Dataframe['Above Range'] = 0
  PD_Dataframe['Bottom of Range'] = round(bottom_of_range, 2)
  PD_Dataframe['Below Range'] = 0

  # Find the columns where a cross happens and updates the column with a 1 if there is a cross
  PD_Dataframe['Above Range'] = np.where(PD_Dataframe['close'] > round(top_of_range, 2), 1, 0)
  PD_Dataframe['Below Range'] = np.where(PD_Dataframe['close'] < round(bottom_of_range, 2), 1, 0)

  return(PD_Dataframe)

In [25]:
spy_bars = alpaca_api.get_bars('SPY', TimeFrame.Day, '2020-01-01', '2020-12-31').df
spy_bars

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2020-01-02 05:00:00+00:00,323.58,324.89,322.530,324.87,59291031,304890,323.710733
2020-01-03 05:00:00+00:00,321.19,323.64,321.100,322.43,77783401,358502,322.791700
2020-01-06 05:00:00+00:00,320.44,323.73,320.360,323.73,55818808,255768,322.722357
2020-01-07 05:00:00+00:00,323.02,323.54,322.240,322.74,43415360,226065,322.688238
2020-01-08 05:00:00+00:00,322.99,325.78,322.670,324.42,68595620,340007,324.323890
...,...,...,...,...,...,...,...
2020-12-24 05:00:00+00:00,368.06,369.03,367.450,369.03,27097853,134316,368.384500
2020-12-28 05:00:00+00:00,371.81,372.59,371.070,372.22,39000402,217154,371.734365
2020-12-29 05:00:00+00:00,373.78,374.00,370.830,371.62,53778116,281875,371.896637


In [26]:
VolBreakout(spy_bars, 1)

,open,high,low,close,volume,trade_count,vwap,Top of Range,Above Range,Bottom of Range,Below Range
timestamp,,,,,,,,,,,
2020-01-02 05:00:00+00:00,323.58,324.89,322.530,324.87,59291031,304890,323.710733,327.33,0,319.87,0
2020-01-03 05:00:00+00:00,321.19,323.64,321.100,322.43,77783401,358502,322.791700,324.94,0,317.48,0
2020-01-06 05:00:00+00:00,320.44,323.73,320.360,323.73,55818808,255768,322.722357,324.19,0,316.73,0
2020-01-07 05:00:00+00:00,323.02,323.54,322.240,322.74,43415360,226065,322.688238,326.77,0,319.31,0
2020-01-08 05:00:00+00:00,322.99,325.78,322.670,324.42,68595620,340007,324.323890,326.74,0,319.28,0
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24 05:00:00+00:00,368.06,369.03,367.450,369.03,27097853,134316,368.384500,371.81,0,364.35,0
2020-12-28 05:00:00+00:00,371.81,372.59,371.070,372.22,39000402,217154,371.734365,375.56,0,368.10,0
2020-12-29 05:00:00+00:00,373.78,374.00,370.830,371.62,53778116,281875,371.896637,377.53,0,370.07,0
